In [1]:
import random as rnd
import string 
from tqdm import tqdm
import math
import mmh3

In [2]:
def gen_rnd_string(len: int)-> str:
    """
    This function generate random string with lenght = len.
    """
    return "".join(rnd.choice(string.ascii_letters + string.digits) for _ in range(len))

In [3]:
def list_to_hash(bloom: list)->int:
    """
    This function hash list with 0 and 1 to int.
    """
    hash = 0
    two_pow = 1
    for i in bloom:
        hash += two_pow * i
        two_pow *= 2

    return hash

генерируем полбзователей и их бакеты с адрессами и токены на которые указывают аддреса

In [4]:
t = 1000 # кол-во пользователей
r = 100 # кол-во аддресов в бакете

address = [] # все адреса
tokens = [] # все токены, пока просто строки длины 100
address_token_map = dict() 

for i in tqdm(range(t)):
    # пока каждый user имеет только один бакет с адресами
    usesr_address = []
    users_token = []

    for j in range(r):
        cur_addres = int(rnd.uniform(1, 1_000_000_000))
        cur_token = gen_rnd_string(100)
        usesr_address.append(cur_addres)
        users_token.append(cur_token)
        address_token_map[cur_addres] = cur_token

    address.append(usesr_address)
    tokens.append(users_token)


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:04<00:00, 226.16it/s]


In [5]:
m = 5000 # размер маски
n = 1000 # желаемый размер ответа
k = 22 # кол-во хеш функций

blooms = [] # массив масок всех адресов
blooms_token_map = dict()
address_bloom_map = dict()
token_bloom_map = dict()

for i in tqdm(range(t)):
    for o, cur_addres in enumerate(address[i]):
        cur_bloom = [0]*m
        cur_token = tokens[i][o]
        for j in range(k):
            nonce = f'nonce_{j}'
            index = mmh3.hash128(str(cur_addres) + nonce) % m
            cur_bloom[index] = 1
        blooms.append(cur_bloom)
        blooms_token_map[list_to_hash(cur_bloom)] = cur_token
        token_bloom_map[cur_token] = cur_bloom
        address_bloom_map[cur_addres] = cur_bloom

100%|██████████| 1000/1000 [01:45<00:00,  9.44it/s]


In [6]:
def find_l(N):
    """
    This function find l parametr(count of added 1 in mask) from N - size of data base, m, k, and r to get n element by query.
    """
    return math.log(1 - (n/N)**(1/k))/math.log(1-1/m) - r*k

In [7]:
def gen_user_bloom(user_number, N):
    """
    This function generate user mask of bucket of address.
    """
    user_bloom = [0]*m

    for cur_addres in address[user_number]:
        cur_bloom = address_bloom_map[cur_addres]
        for ind, val in enumerate(cur_bloom):
            if val == 1:
                user_bloom[ind] = 1
    
    l = find_l(N)
    print('l = ' + str(int(l)))

    for i in range(int(l)):
        index = int(rnd.uniform(0, m))
        user_bloom[index] = 1

    print('user bloom is finished!')
    return user_bloom
    

In [8]:
def find(user_number, N):
    """
    This function find all address in data base, which in user mask. 
    """
    ans = []
    user_bloom = gen_user_bloom(user_number, N)

    for j in tqdm(range(N)):
        bloom = blooms[j]
        flag = True
        for i, val in enumerate(bloom):
            if val == 1 and user_bloom[i] == 0:
                flag = False
                break
        
        if flag:
            ans.append(blooms_token_map[list_to_hash(bloom)])

    print('server find is finished')
    return ans

In [9]:
def check_true(user_number, N):
    """
    This function print size server answer and check that answer has all other tokens.
    """
    server_ans = find(user_number, N)

    print('count returned: ' + str(len(server_ans)))

    for token in tqdm(tokens[user_number]):
        if token in server_ans:
            continue
        else:
            return False

    
    return True

In [10]:
for i in [2, 10, 20, 50, 54]:
    res = check_true(i, 50000)
    print("algo is: " + str(res))

# Провели несколько тестов и оценили матож ответ оклко 1000 как раз!
# статистика будет позже добалена.

l = 6871
user bloom is finished!


100%|██████████| 50000/50000 [00:06<00:00, 8005.15it/s] 


server find is finished
count returned: 1149


100%|██████████| 100/100 [00:00<00:00, 483214.75it/s]


algo is: True
l = 6871
user bloom is finished!


...

algo is: True
l = 6871
user bloom is finished!


100%|██████████| 50000/50000 [00:03<00:00, 12655.39it/s]


server find is finished
count returned: 1044


100%|██████████| 100/100 [00:00<00:00, 323135.90it/s]

algo is: True
